In [77]:
#ライブラリをインポート
import os #OSに依存する様々な機能を利用するためのモジュール(ファイルやディレクトリ操作など)
import re #正規表現を利用するためのモジュール
import csv  #csvファイルを扱うためのモジュール
import math #数学的計算のためのモジュール
import cv2 #画像処理のためのモジュール
import matplotlib.pyplot as plt #グラフ描画のためのモジュール
import numpy as np  #多次元配列計算のためのモジュール
import pandas as pd #データフレームを扱うためのモジュール
from scipy import signal  #信号処理のためのモジュール
from scipy.stats import skew, kurtosis  #歪度と尖度を調べるためのモジュール
from sklearn.model_selection import train_test_split  #データをトレーニング用とテスト用に分けるためのモジュール
from sklearn import preprocessing #データを正規化するためのモジュール
from sklearn.preprocessing import StandardScaler  #データを標準化するためのモジュール
from sklearn.preprocessing import LabelEncoder  #カテゴリ変数を数値化するためのモジュール
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error, r2_score #機械学習モデルの性能評価のためのモジュール
import tensorflow as tf #TensorFlow(Googleが開発したオープンソースの機械学習フレームワーク)
from tensorflow import keras  #TensorFlow用のニューラルネットワークライブラリAPI
from tensorflow.keras import layers #ニューラルネットワークのレイヤーを定義するためのモジュール

In [78]:
#メモ
#画像のサイズは縦：横=1088:1920=17:30

In [79]:
#自分のGoogle Driveとドッキング
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [80]:
#GPUを使うための処理
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [81]:
#正解データへのパスとファイル名取得
CollectPath = "/content/drive/MyDrive/DICOMO/正解データ/"
CollectFilename = os.listdir(CollectPath)

In [82]:
#不正解データへのパスとファイル名取得
IncollectPath = "/content/drive/MyDrive/DICOMO/不正解データ/"
IncollectFilename = os.listdir(IncollectPath)

In [97]:
#教師データを格納するリストを定義
img = []

In [98]:
#正解データをグレースケールに変換・縦170横300にリサイズ
for i in CollectFilename:
     image = cv2.imread(CollectPath+i)
     image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
     image = cv2.resize(image, dsize=(300, 170))
     img.append(image)

In [99]:
#不正解データをグレースケールに変換・縦170横300にリサイズ
for i in CollectFilename:
     image = cv2.imread(IncollectPath+i)
     image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
     image = cv2.resize(image, dsize=(300, 170))
     img.append(image)

error: ignored

In [86]:
#教師ラベルを作成
#渡れる時を1、渡れない時を0とする
CollectLabel = np.ones(len(CollectFilename))
IncollectLabel = np.zeros(len(IncollectFilename))
label = np.concatenate((CollectLabel, IncollectLabel))

In [87]:
#マクロの定義
N = 1 #Encoderの層の数
NUM_HEADS =  16
KEY_DIM = 600
DROPOUT = 0.1 #ドロップアウト率

In [88]:
#トレーニングデータとテストデータの分割
x_train, x_test, y_train, y_test = train_test_split(img, label, train_size = 0.8, shuffle = True)

ValueError: ignored

In [ ]:
# Define the input shape
input_shape = (170, 300)
output_shape = (1,)

#形を定義(このモジュールは行列でないとダメっぽい)
inputs_encoder = layers.Input(shape=input_shape)
inputs_decoder = layers.Input(shape=output_shape)

In [ ]:
#Encoderに対する入力の形状
x_encoder = layers.Reshape((85, 600))(inputs_encoder)

In [ ]:
#Transformer Encoder Layer(BERT)
for i in range(N):
      #Multi-Head-Attention Layer
      attention_encoder = layers.MultiHeadAttention(num_heads=NUM_HEADS, key_dim=KEY_DIM, use_bias=True)(x_encoder, x_encoder, x_encoder)

      #Dropout Layer
      attention_encoder = layers.Dropout(rate=DROPOUT)(attention_encoder)
      #Add & Norm Layer
      attention_encoder = layers.LayerNormalization()(x_encoder + attention_encoder)

      #Feed-Forward-Network
      ffn_encoder = layers.Dense(600 * 4, use_bias=True, activation="relu")(attention_encoder)
      ffn_encoder = layers.Dense(600, use_bias=True)(ffn_encoder)

      #Dropout Layer
      ffn_encoder = layers.Dropout(rate=DROPOUT)(ffn_encoder)
      #Add & Norm Layer
      x_encoder = layers.LayerNormalization()(attention_encoder + ffn_encoder)

In [ ]:
x = layers.Flatten()(x_encoder)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(64, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

In [ ]:
model = keras.Model(inputs=inputs_encoder, outputs=outputs)

In [ ]:
# compile model
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
    )

In [ ]:
model.fit(x_train, y_train, batch_size=32, epochs=300, shuffle=True, validation_split=0.2)
y_pred = model.predict(x_test)

In [ ]:
accuracy_score(y_pred, y_test)